In [1]:
# =====================================================
# FAKE NEWS DETECTION – LSTM + GonzaloA/fake_news
# (Baseline Deep Learning - TensorFlow/Keras Implementation)
# =====================================================

# 1. CÀI ĐẶT & IMPORT
# !pip install -q datasets pandas numpy tensorflow scikit-learn

import re
import time
import warnings
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# CẤU HÌNH HYPERPARAMETERS CHO LSTM
MAX_NB_WORDS = 50000      # Giới hạn từ vựng (Top 50k từ)
MAX_SEQUENCE_LENGTH = 300 # Độ dài tối đa mỗi bài báo (cắt hoặc pad)
EMBEDDING_DIM = 100       # Kích thước vector embedding
BATCH_SIZE = 64
EPOCHS = 5                # LSTM thường hội tụ nhanh

# 2. TẢI DATASET
print("\nĐang tải dataset GonzaloA/fake_news...")
dataset = load_dataset("GonzaloA/fake_news")

# 3. XỬ LÝ DỮ LIỆU
def process_data(ds_split):
    df = pd.DataFrame(ds_split)
    # Ghép title + text
    df['content'] = df.get('title', '').fillna('') + " " + df.get('text', '').fillna('')
    return df

if 'validation' in dataset and 'test' in dataset:
    print("Sử dụng các tập train/val/test có sẵn.")
    train_df = process_data(dataset['train'])
    val_df = process_data(dataset['validation'])
    test_df = process_data(dataset['test'])
else:
    print("Tự chia tập dữ liệu...")
    df = process_data(dataset['train'])
    # Split train/temp (80/20)
    train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
    # Split temp -> val/test (10/10)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Hàm làm sạch văn bản
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # Chỉ giữ chữ cái và số
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("Đang làm sạch văn bản...")
train_df['clean_content'] = train_df['content'].apply(clean_text)
val_df['clean_content'] = val_df['content'].apply(clean_text)
test_df['clean_content'] = test_df['content'].apply(clean_text)

# Lọc các mẫu quá ngắn (< 20 ký tự)
train_df = train_df[train_df['clean_content'].str.len() > 20]
val_df = val_df[val_df['clean_content'].str.len() > 20]
test_df = test_df[test_df['clean_content'].str.len() > 20]

print(f"Sizes -> Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# 4. TOKENIZER & PADDING
print("\nĐang Tokenize văn bản...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_df['clean_content'].values)
word_index = tokenizer.word_index
print(f'Tìm thấy {len(word_index)} từ vựng độc nhất.')

def tokenize_and_pad(df_series, tokenizer):
    sequences = tokenizer.texts_to_sequences(df_series.values)
    padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return padded

X_train = tokenize_and_pad(train_df['clean_content'], tokenizer)
X_val = tokenize_and_pad(val_df['clean_content'], tokenizer)
X_test = tokenize_and_pad(test_df['clean_content'], tokenizer)

y_train = train_df['label'].values
y_val = val_df['label'].values
y_test = test_df['label'].values

print("Shape of data tensor:", X_train.shape)
print("Shape of label tensor:", y_train.shape)

# 5. XÂY DỰNG MÔ HÌNH LSTM
print("\nXây dựng mô hình LSTM...")
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2)) # Dropout tốt cho Embedding
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.0)) # recurrent_dropout=0 để dùng được CuDNN (nhanh hơn trên GPU)
model.add(Dense(1, activation='sigmoid')) # Binary classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# 6. HUẤN LUYỆN
print("\nBắt đầu huấn luyện...")
# EarlyStopping: Dừng nếu val_loss không giảm sau 3 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001, restore_best_weights=True)

start_train_time = time.time()
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)
train_time = time.time() - start_train_time
print(f"Tổng thời gian train: {train_time:.2f}s")

# 7. ĐÁNH GIÁ & ĐO THỜI GIAN (Format chuẩn Trainer)
print("\nĐang đánh giá trên tập Test...")

start_eval_time = time.time()
# Dự đoán xác suất (cho AUC)
y_prob = model.predict(X_test, verbose=0)
# Chuyển về nhãn 0/1 (ngưỡng 0.5)
y_pred = (y_prob > 0.5).astype(int).flatten()
end_eval_time = time.time()

# Tính toán các chỉ số thời gian
eval_runtime = end_eval_time - start_eval_time
n_samples = len(y_test)
eval_samples_per_second = n_samples / eval_runtime

# Tính toán các chỉ số hiệu năng
acc = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
auc = roc_auc_score(y_test, y_prob)

# Lấy loss trên tập test
test_loss = model.evaluate(X_test, y_test, verbose=0)[0]

# Tạo dictionary kết quả (giống Hugging Face Trainer output)
results = {
    'eval_accuracy': acc,
    'eval_precision': precision,
    'eval_recall': recall,
    'eval_f1': f1,
    'eval_auc': float(auc),
    'eval_loss': test_loss,
    'eval_runtime': eval_runtime,
    'eval_samples_per_second': eval_samples_per_second,
    'eval_steps_per_second': eval_samples_per_second / BATCH_SIZE # Ước lượng
}

print("-" * 30)
print("LSTM_GonzaloA_FakeNews_Results")
print(results)
print("-" * 30)



<>:82: SyntaxWarning: invalid escape sequence '\]'
<>:82: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-423276738.py:82: SyntaxWarning: invalid escape sequence '\]'
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)



Đang tải dataset GonzaloA/fake_news...


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json: 0.00B [00:00, ?B/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None

Bắt đầu huấn luyện...
Epoch 1/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8768 - loss: 0.2975 - val_accuracy: 0.9636 - val_loss: 0.1111
Epoch 2/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9615 - loss: 0.1086 - val_accuracy: 0.9465 - val_loss: 0.1487
Epoch 3/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9771 - loss: 0.0653 - val_accuracy: 0.9655 - val_loss: 0.1095
Epoch 4/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9892 - loss: 0.0342 - val_accuracy: 0.9644 - val_loss: 0.1367
Epoch 5/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9942 - loss: 0.0193 - val_accuracy: 0.9570 - val_loss: 0.1548
Tổng thời gian train: 46.50s

Đang đánh giá trên tập Test...
------------------------------
LSTM_GonzaloA_FakeNews_Results
{'eval_accuracy': 0.962794135764445, 'eval_precision': 0.9628826037840472, 'eval_recall': 0.962794135764445, 'eval_f1': 0.9628073573574218, 'eval_auc': 0.9920344166534004, 'eval_loss': 0.11386843025684357, 'e